In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from libtiff import TIFF
import cv2

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

# Generate an initial image with two overlapping circles
tif = TIFF.open('t100.tif',mode='r')
image = tif.read_image()
image1 = image
#image = cv2.bilateralFilter(image,9,75,75)
#image = cv2.imread('t001.tif',cv2.IMREAD_UNCHANGED)
#image = cv2.medianBlur(image, 5)

sobelX = cv2.Sobel(image,cv2.CV_64F,1,0)#x方向的梯度
sobelY = cv2.Sobel(image,cv2.CV_64F,0,1)#y方向的梯度
sobelX = np.uint8(np.absolute(sobelX))#x方向梯度的绝对值
sobelY = np.uint8(np.absolute(sobelY))#y方向梯度的绝对值
image = cv2.bitwise_or(sobelX,sobelY)#
ret, image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#kernel = np.ones((3,3), np.uint8)
#image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
#image = cv2.GaussianBlur(image,(9,9),0)
image = cv2.dilate(image, np.ones((11,11), np.uint8))
image = cv2.erode(image, np.ones((15,15), np.uint8))

# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt(image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
                            labels=image)
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=image)

fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image1, cmap=plt.cm.gray)
ax[0].set_title('Objects')
ax[1].imshow(-distance, cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()
